# Chapter 21: Utility Types

---

## 21.1 Overview of Utility Types

TypeScript provides a rich set of built-in utility types that perform common type transformations. These utilities are built using mapped types, conditional types, and template literals that we've covered in previous chapters. Understanding both how to use them and how they're implemented is essential for advanced TypeScript development.

### 21.1.1 The Built-in Type Toolbox

Utility types are globally available in TypeScript without requiring any imports. They solve recurring type transformation patterns that appear in almost every TypeScript codebase.

```typescript
// Without utility types, you'd write verbose type definitions
interface User {
  id: number;
  name: string;
  email: string;
  age: number;
}

// Creating variations manually is error-prone
interface PartialUser {
  id?: number;
  name?: string;
  email?: string;
  age?: number;
}

interface UserPreview {
  id: number;
  name: string;
  // Omit email and age manually
}

// With utility types, these become one-liners
type PartialUser = Partial<User>;
type UserPreview = Pick<User, "id" | "name">;
type UserUpdate = Omit<User, "id">;
```

**Categories of Utility Types:**

```
┌─────────────────────────────────────────────────────────────────────┐
│                    TypeScript Utility Types                          │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   Property Modifiers:                                               │
│   • Partial<T>    - Make all properties optional                    │
│   • Required<T>   - Make all properties required                    │
│   • Readonly<T>   - Make all properties readonly                    │
│                                                                     │
│   Object Transformations:                                           │
│   • Record<K, T>  - Create object type with keys K and values T    │
│   • Pick<T, K>    - Select specific keys from T                     │
│   • Omit<T, K>    - Remove specific keys from T                     │
│                                                                     │
│   Union Manipulation:                                               │
│   • Exclude<T, U> - Remove types from T that are assignable to U   │
│   • Extract<T, U> - Extract types from T that are assignable to U  │
│   • NonNullable<T> - Remove null and undefined from T               │
│                                                                     │
│   Function Introspection:                                           │
│   • Parameters<T>  - Extract parameter types as tuple               │
│   • ReturnType<T>  - Extract return type                            │
│   • InstanceType<T> - Extract instance type from constructor        │
│                                                                     │
│   String Manipulation:                                              │
│   • Uppercase<T>   - Convert to uppercase                           │
│   • Lowercase<T>   - Convert to lowercase                           │
│   • Capitalize<T>  - Capitalize first letter                        │
│   • Uncapitalize<T> - Lowercase first letter                        │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

### 21.1.2 How Utility Types Are Built

Understanding the implementation of utility types helps you create your own custom variants and debug type issues.

```typescript
// Most utility types are implemented using mapped types and conditionals

// Partial<T> implementation
type MyPartial<T> = {
  [P in keyof T]?: T[P];
};

// Required<T> implementation
type MyRequired<T> = {
  [P in keyof T]-?: T[P];  // -? removes optionality
};

// Readonly<T> implementation
type MyReadonly<T> = {
  readonly [P in keyof T]: T[P];
};

// Record<K, T> implementation
type MyRecord<K extends keyof any, T> = {
  [P in K]: T;
};

// Pick<T, K> implementation
type MyPick<T, K extends keyof T> = {
  [P in K]: T[P];
};

// Omit<T, K> implementation
type MyOmit<T, K extends keyof any> = Pick<T, Exclude<keyof T, K>>;

// Extract<T, U> implementation
type MyExtract<T, U> = T extends U ? T : never;

// Exclude<T, U> implementation
type MyExclude<T, U> = T extends U ? never : T;

// NonNullable<T> implementation
type MyNonNullable<T> = T extends null | undefined ? never : T;
```

---

## 21.2 Property Modifiers

Property modifier utilities change the optionality or mutability of object properties. These are among the most frequently used utilities in TypeScript development.

### 21.2.1 `Partial<T>`

Makes all properties in `T` optional. Useful when you want to allow objects with a subset of properties, such as for updates or partial configurations.

```typescript
interface User {
  id: number;
  name: string;
  email: string;
  age: number;
}

// All properties become optional
type PartialUser = Partial<User>;
// {
//   id?: number;
//   name?: string;
//   email?: string;
//   age?: number;
// }

// Use case: Update functions
function updateUser(userId: number, updates: Partial<User>): User {
  const existingUser = getUser(userId);
  return { ...existingUser, ...updates };
}

// Valid calls - provide any subset of properties
updateUser(1, { name: "John" });
updateUser(1, { email: "new@example.com", age: 30 });
updateUser(1, {}); // Empty object is valid

// Deep Partial (custom implementation)
type DeepPartial<T> = {
  [P in keyof T]?: T[P] extends object ? DeepPartial<T[P]> : T[P];
};

interface Config {
  server: {
    host: string;
    port: number;
  };
}

const partialConfig: DeepPartial<Config> = {
  server: {
    host: "localhost" // port is optional even in nested object
  }
};
```

### 21.2.2 `Required<T>`

Makes all properties in `T` required, removing optionality (`?`). The opposite of `Partial<T>`.

```typescript
interface UserSettings {
  theme?: string;
  notifications?: boolean;
  language?: string;
}

// All properties become required
type RequiredSettings = Required<UserSettings>;
// {
//   theme: string;
//   notifications: boolean;
//   language: string;
// }

// Use case: Ensuring complete configuration
function initializeApp(settings: RequiredSettings): void {
  // TypeScript guarantees all properties exist
  console.log(settings.theme);        // No error
  console.log(settings.notifications); // No error
  console.log(settings.language);      // No error
}

// Combining with Partial for progressive enhancement
type Stage1Config = Partial<DatabaseConfig>;
type Stage2Config = Required<Pick<DatabaseConfig, "host" | "port">> & 
                    Partial<Omit<DatabaseConfig, "host" | "port">>;

interface DatabaseConfig {
  host: string;
  port: number;
  username?: string;
  password?: string;
  database?: string;
}

// Require specific fields while keeping others optional
type StrictDatabaseConfig = Required<Pick<DatabaseConfig, "host" | "port">> & 
                           Partial<Pick<DatabaseConfig, "username" | "password" | "database">>;

const config: StrictDatabaseConfig = {
  host: "localhost",  // Required
  port: 5432,         // Required
  // username is optional
};
```

### 21.2.3 `Readonly<T>`

Makes all properties in `T` readonly, preventing reassignment after initialization.

```typescript
interface Point {
  x: number;
  y: number;
}

// All properties become readonly
type ReadonlyPoint = Readonly<Point>;
// {
//   readonly x: number;
//   readonly y: number;
// }

const point: ReadonlyPoint = { x: 10, y: 20 };
// point.x = 30; // ❌ Error: Cannot assign to 'x' because it is a read-only property

// Use case: Immutable data structures
interface State {
  users: User[];
  loading: boolean;
}

const initialState: Readonly<State> = {
  users: [],
  loading: false
};

// State updates must create new objects
function reducer(state: State, action: Action): State {
  switch (action.type) {
    case "ADD_USER":
      return {
        ...state,
        users: [...state.users, action.payload]
      };
    default:
      return state;
  }
}

// Deep Readonly (custom implementation)
type DeepReadonly<T> = {
  readonly [P in keyof T]: T[P] extends object ? DeepReadonly<T[P]> : T[P];
};

interface ComplexState {
  user: {
    profile: {
      name: string;
    };
  };
}

const frozenState: DeepReadonly<ComplexState> = {
  user: {
    profile: {
      name: "John"
    }
  }
};

// frozenState.user.profile.name = "Jane"; // ❌ Error at all levels
```

**Property Modifier Interaction:**

```typescript
// You can chain property modifiers
interface User {
  id: number;
  name?: string;
  email?: string;
}

// Make all required, then make all readonly
type ReadonlyRequiredUser = Readonly<Required<User>>;
// {
//   readonly id: number;
//   readonly name: string;
//   readonly email: string;
// }

// Remove readonly with mapped type
type Mutable<T> = {
  -readonly [P in keyof T]: T[P];
};

// Remove optionality
type Definitely<T> = {
  [P in keyof T]-?: T[P];
};
```

---

## 21.3 Type Transformations

These utilities transform object types by selecting, omitting, or creating keys. They are essential for deriving types from existing types without repetition.

### 21.3.1 `Record<K, T>`

Constructs an object type with keys of type `K` and values of type `T`. Perfect for dictionaries, maps, and lookup tables.

```typescript
// Basic usage
type PageNames = "home" | "about" | "contact";
type PageInfo = { title: string; path: string };

const pages: Record<PageNames, PageInfo> = {
  home: { title: "Home", path: "/" },
  about: { title: "About", path: "/about" },
  contact: { title: "Contact", path: "/contact" }
};

// Without Record, you'd write:
interface Pages {
  home: PageInfo;
  about: PageInfo;
  contact: PageInfo;
}

// Dynamic keys with string
type StringDictionary = Record<string, number>;
const scores: StringDictionary = {
  alice: 100,
  bob: 85,
  charlie: 92
};

// Enum as keys
enum Status {
  Pending = "PENDING",
  Approved = "APPROVED",
  Rejected = "REJECTED"
}

type StatusHandlers = Record<Status, () => void>;

const handlers: StatusHandlers = {
  [Status.Pending]: () => console.log("Waiting..."),
  [Status.Approved]: () => console.log("Approved!"),
  [Status.Rejected]: () => console.log("Rejected")
};

// Partial Record (some keys optional)
type PartialRecord<K extends keyof any, T> = {
  [P in K]?: T;
};

// Use case: Caching
const cache: PartialRecord<string, User> = {};
cache["user-1"] = { id: 1, name: "John" };
// cache["user-2"] is undefined (valid)
```

### 21.3.2 `Pick<T, K>`

Selects a subset of properties `K` from type `T`. Useful for creating "views" of larger types.

```typescript
interface User {
  id: number;
  name: string;
  email: string;
  age: number;
  password: string;
  createdAt: Date;
}

// Select only specific properties
type PublicUser = Pick<User, "id" | "name" | "email">;
// {
//   id: number;
//   name: string;
//   email: string;
// }

// Use case: API responses (hide sensitive data)
function getPublicUser(user: User): PublicUser {
  return {
    id: user.id,
    name: user.name,
    email: user.email
    // TypeScript ensures we don't accidentally include password
  };
}

// Dynamic key selection with conditional types
type PickByValueType<T, U> = Pick<T, {
  [K in keyof T]: T[K] extends U ? K : never;
}[keyof T]>;

interface Mixed {
  name: string;
  age: number;
  active: boolean;
  score: number;
}

type NumberProps = PickByValueType<Mixed, number>;
// { age: number; score: number }

// Pick with template literals
interface APIResponse {
  "data:users": User[];
  "data:orders": Order[];
  "meta:total": number;
  "meta:page": number;
}

type DataKeys = keyof APIResponse extends infer K 
  ? K extends `data:${infer _}` ? K : never 
  : never;
// "data:users" | "data:orders"

type DataOnly = Pick<APIResponse, DataKeys>;
```

### 21.3.3 `Omit<T, K>`

Removes properties `K` from type `T`. The opposite of `Pick`.

```typescript
interface User {
  id: number;
  name: string;
  email: string;
  password: string;
  createdAt: Date;
  updatedAt: Date;
}

// Remove specific properties
type UserFormData = Omit<User, "id" | "createdAt" | "updatedAt">;
// {
//   name: string;
//   email: string;
//   password: string;
// }

// Use case: Forms (exclude auto-generated fields)
function createUser(formData: UserFormData): User {
  return {
    ...formData,
    id: generateId(),
    createdAt: new Date(),
    updatedAt: new Date()
  };
}

// Omit with unions
type UpdateUserData = Omit<User, "id" | "createdAt" | "updatedAt">;
// All fields optional for updates
type PartialUpdate = Partial<UpdateUserData>;

// StrictOmit (ensure keys exist in T)
type StrictOmit<T, K extends keyof T> = Omit<T, K>;

// This helps catch typos in keys:
// type BadOmit = StrictOmit<User, "idd">; // ❌ Error: "idd" not in User

// DeepOmit (custom implementation)
type DeepOmit<T, K extends keyof any> = T extends object 
  ? {
      [P in keyof T as P extends K ? never : P]: DeepOmit<T[P], K>;
    }
  : T;

interface Nested {
  id: number;
  user: {
    id: number;
    name: string;
    profile: {
      id: number;
      bio: string;
    };
  };
}

type NoIds = DeepOmit<Nested, "id">;
// Removes all 'id' properties at every level
```

---

## 21.4 Union Manipulation

These utilities work with union types to filter, extract, or remove specific members.

### 21.4.1 `Exclude<T, U>`

Removes from `T` all types that are assignable to `U`. Useful for filtering union types.

```typescript
type AllTypes = string | number | boolean | null | undefined;

// Remove null and undefined
type NonNull = Exclude<AllTypes, null | undefined>;
// string | number | boolean

// Remove specific literals
type Status = "pending" | "success" | "error" | "cancelled";
type FinalStatus = Exclude<Status, "pending">;
// "success" | "error" | "cancelled"

// Use case: Event handlers
type Event = 
  | { type: "click"; x: number; y: number }
  | { type: "focus"; element: HTMLElement }
  | { type: "blur"; element: HTMLElement }
  | { type: "custom"; data: any };

type StandardEvents = Exclude<Event, { type: "custom" }>;
// Excludes the custom event type

// Exclude from object keys
type ExcludeKeys<T, K extends keyof T> = Pick<T, Exclude<keyof T, K>>;

interface User {
  id: number;
  name: string;
  password: string;
}

type SafeUser = ExcludeKeys<User, "password">;
// { id: number; name: string }
```

### 21.4.2 `Extract<T, U>`

Extracts from `T` all types that are assignable to `U`. The opposite of `Exclude`.

```typescript
type AllTypes = string | number | boolean | null | undefined;

// Extract specific types
type JustStrings = Extract<AllTypes, string>;
// string (extracts the string type from union)

type Events = 
  | { type: "user"; id: number }
  | { type: "order"; orderId: string }
  | { type: "product"; sku: string }
  | string; // Some legacy events are just strings

// Extract only object events
type ObjectEvents = Extract<Events, { type: any }>;
// { type: "user"; id: number } | { type: "order"; orderId: string } | { type: "product"; sku: string }

// Extract by key
type ExtractByType<T, Type extends string> = Extract<T, { type: Type }>;

type UserEvent = ExtractByType<Events, "user">;
// { type: "user"; id: number }

// Extract functions from mixed array
type MixedArray = (() => void) | string | number | (() => string);
type Functions = Extract<MixedArray, Function>;
// (() => void) | (() => string)
```

### 21.4.3 `NonNullable<T>`

Removes `null` and `undefined` from type `T`. Equivalent to `Exclude<T, null | undefined>`.

```typescript
type MaybeUser = User | null | undefined;

// Remove null and undefined
type DefinitelyUser = NonNullable<MaybeUser>;
// User

// Use case: Array filtering
const users: (User | null)[] = fetchUsers();

// Type guard to filter nulls
function isNotNull<T>(value: T | null): value is NonNullable<T> {
  return value !== null;
}

const validUsers = users.filter(isNotNull);
// Type: User[] (nulls removed)

// Strict NonNullable (custom)
type StrictNonNullable<T> = T extends null | undefined ? never : T;

// Deep NonNullable
type DeepNonNullable<T> = T extends object 
  ? { [K in keyof T]: DeepNonNullable<T[K]> }
  : NonNullable<T>;
```

---

## 21.5 Function Types

These utilities extract type information from function types, enabling type-safe higher-order functions and introspection.

### 21.5.1 `Parameters<T>`

Extracts the parameter types of a function type as a tuple.

```typescript
declare function createUser(
  name: string, 
  email: string, 
  age: number
): User;

// Extract parameters as tuple
type CreateUserParams = Parameters<typeof createUser>;
// [name: string, email: string, age: number]

// Use case: Type-safe wrappers
function logWrapper<T extends (...args: any[]) => any>(
  fn: T,
  ...args: Parameters<T>
): ReturnType<T> {
  console.log(`Calling with args:`, args);
  return fn(...args);
}

logWrapper(createUser, "John", "john@example.com", 30); // ✅ Type-safe!

// Extract specific parameter
type FirstParameter<T extends (...args: any[]) => any> = 
  Parameters<T> extends [infer P, ...any[]] ? P : never;

type FirstArg = FirstParameter<typeof createUser>; // string

// Parameters with rest
declare function logMessages(...messages: string[]): void;
type LogParams = Parameters<typeof logMessages>; // [messages: ...string[]]
```

### 21.5.2 `ConstructorParameters<T>`

Extracts the parameter types of a constructor function type as a tuple.

```typescript
class User {
  constructor(
    public name: string,
    public email: string,
    public age: number
  ) {}
}

// Extract constructor parameters
type UserConstructorParams = ConstructorParameters<typeof User>;
// [name: string, email: string, age: number]

// Use case: Factory functions
function createUserFactory(
  ...args: ConstructorParameters<typeof User>
): User {
  return new User(...args);
}

// Works with built-in constructors too
type DateConstructorParams = ConstructorParameters<typeof Date>;
// [] | [value: string | number | Date] | [year: number, month: number, date?: number, ...]

// Extract instance type
type UserInstance = InstanceType<typeof User>; // User
```

### 21.5.3 `ReturnType<T>`

Extracts the return type of a function type.

```typescript
declare function fetchUser(id: number): Promise<User>;

// Extract return type
type FetchUserReturn = ReturnType<typeof fetchUser>;
// Promise<User>

// Unwrap Promise
type UnwrapPromise<T> = T extends Promise<infer U> ? U : T;
type UserData = UnwrapPromise<ReturnType<typeof fetchUser>>;
// User

// Use case: API response types
async function getProducts() {
  return [{ id: 1, name: "Widget" }, { id: 2, name: "Gadget" }];
}

type Product = UnwrapPromise<ReturnType<typeof getProducts>>[number];
// { id: number; name: string }

// Return type of overloaded functions (gets last overload)
function process(x: string): string;
function process(x: number): number;
function process(x: string | number): string | number {
  return x;
}

type ProcessReturn = ReturnType<typeof process>; // number (last overload)
```

### 21.5.4 `InstanceType<T>`

Extracts the instance type from a constructor function type.

```typescript
class Service {
  private data: string[] = [];
  
  getData() {
    return this.data;
  }
}

// Get instance type from class
type ServiceInstance = InstanceType<typeof Service>;
// Service

// Use case: Dependency injection
interface Container {
  get<T extends new (...args: any[]) => any>(
    ctor: T
  ): InstanceType<T>;
}

const container: Container = {
  get<T extends new (...args: any[]) => any>(ctor: T): InstanceType<T> {
    return new ctor();
  }
};

const service = container.get(Service); // Type: Service
```

**Additional Function Utilities:**

```typescript
// ThisParameterType<T> - Extracts the type of 'this' parameter
function greet(this: { name: string }) {
  return `Hello, ${this.name}`;
}

type ThisType = ThisParameterType<typeof greet>;
// { name: string }

// OmitThisParameter<T> - Removes 'this' parameter from type
type GreetFn = OmitThisParameter<typeof greet>;
// () => string

// ThisType<T> - Marker for contextual 'this' type (not a conditional)
type ObjectWithThis = {
  name: string;
  greet(): void;
} & ThisType<{ name: string }>;
```

---

## 21.6 Other Utility Types

Beyond the commonly used utilities, TypeScript provides specialized types for specific scenarios.

### 21.6.1 `ThisParameterType<T>`

Extracts the type of the `this` parameter from a function type, or `unknown` if the function has no `this` parameter.

```typescript
function clickHandler(this: HTMLElement, event: MouseEvent): void {
  console.log(this.tagName);
}

type ClickThis = ThisParameterType<typeof clickHandler>;
// HTMLElement

function regularFunction(a: number): number {
  return a;
}

type RegularThis = ThisParameterType<typeof regularFunction>;
// unknown

// Use case: Binding context
function bindContext<T extends (this: any, ...args: any[]) => any>(
  fn: T,
  context: ThisParameterType<T>
): (...args: Parameters<T>) => ReturnType<T> {
  return fn.bind(context);
}

const boundHandler = bindContext(clickHandler, document.body);
// boundHandler doesn't require 'this' parameter anymore
```

### 21.6.2 `OmitThisParameter<T>`

Removes the `this` parameter from a function type, returning a function type with the same parameters and return type but without the `this` parameter.

```typescript
function method(this: { value: number }, x: number): number {
  return this.value + x;
}

type BoundMethod = OmitThisParameter<typeof method>;
// (x: number) => number

// Use case: Preparing methods for callbacks
class Counter {
  private count = 0;
  
  increment(this: this): void {
    this.count++;
  }
  
  getMethod(): OmitThisParameter<typeof this.increment> {
    return this.increment.bind(this);
  }
}

const counter = new Counter();
const inc = counter.getMethod();
inc(); // Works without context
```

### 21.6.3 `ThisType<T>`

A marker interface used to specify the type of `this` within an object literal. Unlike other utilities, it doesn't transform types but provides contextual typing.

```typescript
// ThisType is used in object method definitions
type ObjectDescriptor<D, M> = {
  data?: D;
  methods?: M & ThisType<D & M>;  // Methods have 'this' typed as D & M
};

function makeObject<D, M>(desc: ObjectDescriptor<D, M>): D & M {
  const data: object = desc.data || {};
  const methods: object = desc.methods || {};
  return { ...data, ...methods } as D & M;
}

const obj = makeObject({
  data: { x: 0, y: 0 },
  methods: {
    moveBy(dx: number, dy: number) {
      this.x += dx;  // 'this' is typed as { x: number, y: number } & methods
      this.y += dy;
    }
  }
});

obj.x = 10;        // OK
obj.moveBy(5, 5);  // OK
```

---

## 21.7 Creating Custom Utility Types

Understanding how to build your own utility types allows you to create domain-specific type transformations that aren't covered by the built-in utilities.

### 21.7.1 Deep Variants

Create deep versions of the built-in utilities for nested objects.

```typescript
// DeepPartial - makes all properties optional recursively
type DeepPartial<T> = T extends object 
  ? { [P in keyof T]?: DeepPartial<T[P]> }
  : T;

// DeepRequired - makes all properties required recursively  
type DeepRequired<T> = T extends object 
  ? { [P in keyof T]-?: DeepRequired<T[P]> }
  : T;

// DeepReadonly - makes all properties readonly recursively
type DeepReadonly<T> = T extends object 
  ? { readonly [P in keyof T]: DeepReadonly<T[P]> }
  : T;

// DeepMutable - removes readonly recursively
type DeepMutable<T> = {
  -readonly [P in keyof T]: T[P] extends object ? DeepMutable<T[P]> : T[P];
};

// Usage
interface Config {
  server: {
    host: string;
    port: number;
    ssl: {
      enabled: boolean;
      cert?: string;
    };
  };
}

const partial: DeepPartial<Config> = {
  server: {
    ssl: {
      enabled: true
      // cert is optional, host/port are optional too
    }
  }
};
```

### 21.7.2 Key Selection Utilities

Create utilities that select keys based on criteria other than just names.

```typescript
// PickByValueType - pick keys whose values extend T
type PickByValueType<T, V> = Pick<T, {
  [K in keyof T]: T[K] extends V ? K : never;
}[keyof T]>;

interface User {
  id: number;
  name: string;
  email: string;
  age: number;
  isActive: boolean;
}

type StringProps = PickByValueType<User, string>;
// { name: string; email: string }

type NumericProps = PickByValueType<User, number>;
// { id: number; age: number }

// PickNullable - pick only optional properties
type PickNullable<T> = Pick<T, {
  [K in keyof T]: undefined extends T[K] ? K : never;
}[keyof T]>;

// PickRequired - pick only required properties
type PickRequired<T> = Pick<T, {
  [K in keyof T]: undefined extends T[K] ? never : K;
}[keyof T]>;
```

### 21.7.3 String Key Manipulation

Transform object keys using template literals.

```typescript
// Add prefix to all keys
type AddKeyPrefix<T, P extends string> = {
  [K in keyof T as `${P}${string & K}`]: T[K];
};

type UserWithPrefix = AddKeyPrefix<User, "user_">;
// { user_id: number; user_name: string; ... }

// Add suffix to all keys
type AddKeySuffix<T, S extends string> = {
  [K in keyof T as `${string & K}${S}`]: T[K];
};

// CamelCase to snake_case keys
type CamelToSnake<S extends string> = S extends `${infer T}${infer U}`
  ? `${T extends Capitalize<T> ? `_${Lowercase<T>}` : T}${CamelToSnake<U>}`
  : S;

type SnakeKeys<T> = {
  [K in keyof T as CamelToSnake<string & K>]: T[K];
};

interface CamelCase {
  userName: string;
  emailAddress: string;
}

type SnakeCase = SnakeKeys<CamelCase>;
// { user_name: string; email_address: string }
```

### 21.7.4 Union and Array Utilities

Utilities for working with unions and arrays.

```typescript
// UnionToIntersection - converts union to intersection
type UnionToIntersection<U> = 
  (U extends any ? (k: U) => void : never) extends ((k: infer I) => void) 
    ? I 
    : never;

type Union = { a: string } | { b: number };
type Intersection = UnionToIntersection<Union>;
// { a: string } & { b: number }

// NonEmptyArray - ensures array has at least one element
type NonEmptyArray<T> = [T, ...T[]];

function firstElement<T>(arr: NonEmptyArray<T>): T {
  return arr[0];
}

firstElement([1, 2, 3]); // ✅ OK
// firstElement([]); // ❌ Error: requires at least one element

// Flatten - flattens array type one level
type Flatten<T extends any[]> = T extends (infer U)[] ? U : never;

type Nested = Flatten<string[][]>; // string[]

// DeepFlatten - flattens completely
type DeepFlatten<T> = T extends (infer U)[]
  ? U extends any[] ? DeepFlatten<U> : U
  : T;
```

---

## 21.8 Chapter Summary and Exercises

### Chapter Summary

In this chapter, we explored TypeScript's built-in utility types and how to create custom ones:

**Key Takeaways:**

1. **Property Modifiers**:
   - `Partial<T>` - Makes all properties optional
   - `Required<T>` - Makes all properties required
   - `Readonly<T>` - Makes all properties readonly
   - Chainable and composable for complex transformations

2. **Object Transformations**:
   - `Record<K, T>` - Creates object with specific key and value types
   - `Pick<T, K>` - Selects specific keys from type
   - `Omit<T, K>` - Removes specific keys from type
   - Essential for creating derived types without duplication

3. **Union Manipulation**:
   - `Exclude<T, U>` - Removes types assignable to U from T
   - `Extract<T, U>` - Keeps only types assignable to U
   - `NonNullable<T>` - Removes null and undefined
   - Critical for filtering union types

4. **Function Introspection**:
   - `Parameters<T>` - Extracts parameter tuple
   - `ReturnType<T>` - Extracts return type
   - `InstanceType<T>` - Extracts instance type from class
   - `ConstructorParameters<T>` - Extracts constructor parameters
   - Enable type-safe higher-order functions

5. **Custom Utilities**:
   - Deep variants for recursive transformations
   - Key selection by value type
   - String key manipulation
   - Union and array utilities

### Practical Exercises

**Exercise 1: Built-in Utilities Practice**

Use built-in utility types to solve these problems:

```typescript
interface Product {
  id: number;
  name: string;
  description: string;
  price: number;
  inStock: boolean;
  category: string;
  createdAt: Date;
  updatedAt: Date;
}

// 1. Create a type for creating products (exclude id and dates)
type CreateProductDTO = /* your code */;

// 2. Create a type for updating products (partial of above, exclude id)
type UpdateProductDTO = /* your code */;

// 3. Create a type for product listings (pick only id, name, price)
type ProductListItem = /* your code */;

// 4. Create a record type for product inventory (id -> quantity)
type Inventory = /* your code */;

// 5. Create a type with only string properties from Product
type ProductStrings = /* your code */;
```

**Exercise 2: Implement Custom Utilities**

Implement these utility types from scratch:

```typescript
// 1. MyPartial<T> - Make all properties optional
type MyPartial<T> = /* implementation */;

// 2. MyPick<T, K> - Select keys K from T
type MyPick<T, K extends keyof T> = /* implementation */;

// 3. MyExclude<T, U> - Exclude U from T
type MyExclude<T, U> = /* implementation */;

// 4. MyParameters<T> - Extract parameters tuple
type MyParameters<T extends (...args: any[]) => any> = /* implementation */;

// 5. MyReturnType<T> - Extract return type
type MyReturnType<T extends (...args: any[]) => any> = /* implementation */;
```

**Exercise 3: Advanced Custom Utilities**

Create advanced utility types:

```typescript
// 1. DeepPartial<T> - Recursive partial
type DeepPartial<T> = /* implementation */;

// 2. PickByValueType<T, V> - Pick keys where value extends V
type PickByValueType<T, V> = /* implementation */;

// 3. RequiredKeys<T> - Get union of required key names
type RequiredKeys<T> = /* implementation */;

// 4. OptionalKeys<T> - Get union of optional key names
type OptionalKeys<T> = /* implementation */;

// 5. Flatten<T> - Flatten nested arrays one level
type Flatten<T> = /* implementation */;
```

**Exercise 4: Real-World Application**

Build a type-safe API client using utility types:

```typescript
// Define API endpoints
interface Endpoints {
  "GET /users": { response: User[]; params: void };
  "GET /users/:id": { response: User; params: { id: string } };
  "POST /users": { response: User; params: { name: string; email: string } };
  "PUT /users/:id": { response: User; params: { id: string } & Partial<User> };
  "DELETE /users/:id": { response: void; params: { id: string } };
}

// Implement:
// 1. Extract endpoint paths as union type
type EndpointPaths = /* "GET /users" | "GET /users/:id" | ... */;

// 2. Create a function type for API client that:
//    - Accepts endpoint path as first argument
//    - Accepts params as second argument (if needed)
//    - Returns Promise of correct response type
declare function apiClient<Path extends EndpointPaths>(
  path: Path,
  params: /* extract params for Path */
): Promise</* extract response for Path */>;

// Test cases:
const users = await apiClient("GET /users"); // User[]
const user = await apiClient("GET /users/:id", { id: "123" }); // User
await apiClient("POST /users", { name: "John", email: "john@example.com" });
```

**Exercise 5: Type Transformation Pipeline**

Create a complex type transformation:

```typescript
interface DatabaseUser {
  user_id: number;
  first_name: string;
  last_name: string;
  email_address: string;
  is_active: boolean;
  created_timestamp: Date;
  updated_timestamp: Date;
  password_hash: string;
  internal_notes: string;
}

// Create a transformation pipeline that:
// 1. Converts snake_case keys to camelCase
// 2. Removes internal fields (password_hash, internal_notes)
// 3. Removes timestamp fields
// 4. Makes all properties optional
// 5. Adds a 'fullName' computed property

type TransformedUser = /* your implementation */;

// Should result in:
// {
//   userId?: number;
//   firstName?: string;
//   lastName?: string;
//   emailAddress?: string;
//   isActive?: boolean;
//   fullName?: string;
// }
```

### Additional Resources

- **TypeScript Handbook - Utility Types**: https://www.typescriptlang.org/docs/handbook/utility-types.html
- **TypeScript Built-in Types**: https://github.com/microsoft/TypeScript/blob/main/src/lib/es5.d.ts (Source code for utility types)
- **Type Challenges**: https://github.com/type-challenges/type-challenges (Practice implementing utility types)
- **Total TypeScript**: https://www.totaltypescript.com/concepts/utility-types (Detailed explanations)

---

## Coming Up Next: Chapter 22 - The `keyof` Operator

In the next chapter, we will explore **The `keyof` Operator**, a fundamental tool for working with object keys at the type level:

- **22.1 Understanding `keyof`** - The basics of key extraction
- **22.2 `keyof` with Generics** - Combining with type parameters for flexibility
- **22.3 `keyof` with Union Types** - Handling unions and intersections
- **22.4 Dynamic Property Access** - Type-safe property access patterns
- **22.5 Type-Safe Object Keys** - Ensuring key safety in functions and mappings

The `keyof` operator is the bridge between values and types, allowing you to extract keys from objects as union types. Combined with utility types, it forms the foundation for building flexible, type-safe APIs and data transformations.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='20. template_literal_types.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='22. the_keyof_operator.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
